# Effect of gap junctions on fast spiking interneurons driven by current injection

This notebook validates the old result that when FS neurons are driven by current injections we will see a synchronisation if there are also gap junctions coupling them together.

Here we create a network of FS neurons, then simulate them without gap junctions ('''gap_junctions_disabled=True'''). Then the gap junctions are manually inserted into the network, and the simulation is rerun this time with the gap junctions. In both cases the FS neurons are driven by current injections (and not synaptic inputs).

In [ ]:
import os
os.environ["SNUDDA_DATA"] = "../../../../BasalGangliaData/data/"
assert os.path.isdir(os.getenv("SNUDDA_DATA")), f"You need to have BasalGangliaData installed for this example. You can run this example without it, but then do not execute this cell."

network_path = os.path.join("FS_network_3-cur-inj")

In [ ]:
from snudda.init.init import SnuddaInit
cnc = SnuddaInit(struct_def={}, network_path=network_path, random_seed=1337)
cnc.define_striatum(num_dSPN=0, num_iSPN=0, num_FS=100, num_LTS=0, num_ChIN=0,
                    volume_type="cube", side_len=230e-6)
cnc.write_json()

In [ ]:
from snudda import SnuddaPlace
sp = SnuddaPlace(network_path=network_path, verbose=False)
sp.place()

In [ ]:
from snudda import SnuddaDetect

sd = SnuddaDetect(network_path=network_path)
sd.detect()

In [ ]:
from snudda import SnuddaPrune

sp = SnuddaPrune(network_path=network_path)
sp.prune()

In [ ]:
# !snudda_load FS_network_2-cur-inj/network-synapses.hdf5 --listN

In [ ]:
!nrnivmodl $SNUDDA_DATA/neurons/mechanisms

### Running FS network without gap junctions (driven by current injections)

In [ ]:
from snudda.simulate.pair_recording import PairRecording 
experiment_config_file = "FS_network_3-experiment-config.json"
output_file_no_gj = os.path.join(network_path, "simulation", "output-no-gj.hdf5")
pr = PairRecording(network_path=network_path, experiment_config_file=experiment_config_file, disable_gap_junctions=True, output_file=output_file_no_gj, verbose=False)

In [ ]:
pr.run()

Note the lack of synchronisation in the network (without gap junctions) in the figure below.

In [ ]:
%matplotlib notebook
pr.plot_trace_overview(experiment_name="FS-cur-inj-without-GJ")

### Adding gap junctions to the network, and rerunning.

In [ ]:
pr.disable_gap_junctions = False
output_file_with_gj = os.path.join(network_path, "simulation", "output-with-gj.hdf5")

pr.output_file = output_file_with_gj
pr.record.set_new_output_file(output_file_with_gj)

# Connect the gap junctions that were disabled previously
pr.connect_network_gap_junctions_local()
pr.pc.setup_transfer()

In [ ]:
pr.run()

The addition of gap junctions to the network synchronises the activity in the network when driven by current injections. It is important to realise that how we drive the network matters, neurons driven by synaptic input would not be synchronised in the same way by gap junctions.

In [ ]:
pr.plot_trace_overview(experiment_name="FS-cur-inj-with-GJ")